In [1]:
#Imports
import numpy as np
import tensorflow as tf
import random as rd

In [2]:
#Importing the dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-02 21:34:01--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  5.10MB/s    in 0.2s    

2023-12-02 21:34:02 (5.10 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
#Read it to inspect it
with open('input.txt','r',encoding='utf-8') as f:
  text=f.read()

In [4]:
print("Length of dataset in characters : ", len(text))

Length of dataset in characters :  1115394


In [5]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
#Mapping des caractères en entiers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s : [stoi[i] for i in s]
decode = lambda s : [itos[i] for i in s]
print(encode('jpt-3'))

[48, 54, 58, 7, 9]


In [8]:
data = np.array(encode(text))
print(data.shape, data.dtype)
print(data[:100])

(1115394,) int64
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]


In [9]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
context_length = 8
train_data[:context_length+1]

array([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:context_length]
y = train_data[1:context_length+1]
for i in range(context_length):
  context=x[:i+1]
  target = y[i]
  print(f'When the input is {context}The output should be {target}')

When the input is [18]The output should be 47
When the input is [18 47]The output should be 56
When the input is [18 47 56]The output should be 57
When the input is [18 47 56 57]The output should be 58
When the input is [18 47 56 57 58]The output should be 1
When the input is [18 47 56 57 58  1]The output should be 15
When the input is [18 47 56 57 58  1 15]The output should be 47
When the input is [18 47 56 57 58  1 15 47]The output should be 58


In [12]:
batch_size = 4
context_length = 8
def get_batch(split):
  data = train_data if split == 'train' else val_data
  list_of_ints = [i for i in range(len(data) - context_length + 1)]
  ix = rd.sample(list_of_ints, batch_size)
  x = np.stack([data[i:i + context_length] for i in ix])  # Utilisation d'une liste en compréhension
  y = np.stack([data[i + 1:i + context_length + 1] for i in ix])  # Idem pour y
  return x, y


xb, yb = get_batch('train')
print('Inputs')
print(xb.shape)
print(xb)
print('Targets')
print(yb.shape)
print(yb)
print('____________________________________')
for b in range(batch_size):
  for t in range(context_length):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"When input is {context.tolist()} the output should be {target}")

Inputs
(4, 8)
[[ 1 39 57 47 42 43  1 58]
 [54 56 43 58 58 63  1 54]
 [ 1 46 47 57  1 42 39 59]
 [43  1 40 43 57 58  2  1]]
Targets
(4, 8)
[[39 57 47 42 43  1 58 46]
 [56 43 58 58 63  1 54 43]
 [46 47 57  1 42 39 59 45]
 [ 1 40 43 57 58  2  1 32]]
____________________________________
When input is [1] the output should be 39
When input is [1, 39] the output should be 57
When input is [1, 39, 57] the output should be 47
When input is [1, 39, 57, 47] the output should be 42
When input is [1, 39, 57, 47, 42] the output should be 43
When input is [1, 39, 57, 47, 42, 43] the output should be 1
When input is [1, 39, 57, 47, 42, 43, 1] the output should be 58
When input is [1, 39, 57, 47, 42, 43, 1, 58] the output should be 46
When input is [54] the output should be 56
When input is [54, 56] the output should be 43
When input is [54, 56, 43] the output should be 58
When input is [54, 56, 43, 58] the output should be 58
When input is [54, 56, 43, 58, 58] the output should be 63
When input is [5

In [13]:
xb, yb = get_batch('train')
print('Inputs')
print(xb.shape)
print(xb)
print('Targets')
print(yb.shape)
print(yb)


Inputs
(4, 8)
[[39 57 59 56 43  8  0  0]
 [63  8  0  0 13 33 18 21]
 [49 43  1 39 52  1 43 62]
 [43 51 40 43 56  1 61 43]]
Targets
(4, 8)
[[57 59 56 43  8  0  0 15]
 [ 8  0  0 13 33 18 21 16]
 [43  1 39 52  1 43 62 43]
 [51 40 43 56  1 61 43 50]]


In [34]:
import tensorflow_probability as tfp

In [43]:
class BigramLanguageModel(tf.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = tf.Variable(tf.random.normal([vocab_size,vocab_size]))
  def __call__(self,idx,targets =None):
    logits = tf.nn.embedding_lookup(self.token_embedding_table,idx)
    if targets is None:
      loss= None
    else :
      B,T,C = logits.shape
      logits = tf.reshape(logits,[B*T,C])
      targets = tf.reshape(targets,[B*T])
      loss = tf.keras.losses.sparse_categorical_crossentropy(targets,logits,from_logits=True)
    return(logits,tf.reduce_mean(loss))
  def generate(self,idx,max_new_tokens):
    for i in range(max_new_tokens):
      logits,loss = self(idx)
      logits = logits [:,-1,:]
      probs = tf.nn.softmax(logits)
      idx_next = tfp.Multinomial(probs=probs,size=1)
      idx = tf.concat([(idx), (idx_next)],axis=1)
      return(idx)

m=BigramLanguageModel(vocab_size)
out=m(xb,yb)
print(out[0].shape)
print(out[1].shape)
print("Logits:", out[0])
print("Loss:", out[1])
print(decode(m.generate(np.zeros(1,dtype=np.int32),max_new_tokens = 100)[0].tolist()))

(32, 65)
()
Logits: tf.Tensor(
[[ 2.2926724  -1.7428583   0.4850148  ... -1.5406622  -1.3667842
  -0.89780337]
 [ 0.6049428  -0.37285373 -0.03189395 ... -0.19215085  0.55007637
   0.34815136]
 [ 0.42745182  0.2451142  -0.82166874 ... -0.90642893 -0.03507071
  -0.24629602]
 ...
 [ 0.22609127 -0.77244824  1.6175609  ... -0.37875894 -2.5461874
   0.4740015 ]
 [-0.80830777 -0.55427396 -0.26638216 ...  0.8952034   0.6210111
  -1.3505417 ]
 [-0.9685111   0.9451213   0.906063   ...  0.7674658   0.20205861
  -2.234185  ]], shape=(32, 65), dtype=float32)
Loss: tf.Tensor(4.5306177, shape=(), dtype=float32)


ValueError: ignored